Setup & Import Library

In [1]:
# %% [markdown]
# # FASE 2 (V2.0): Pembangunan Model Rekomendasi Hybrid
# 
# **Tujuan:**
# 1. Membaca dataset bersih dan model TF-IDF dari fase sebelumnya.
# 2. Memuat model Word2Vec (FastText) Bahasa Indonesia yang sudah di-pre-trained.
# 3. Membuat representasi vektor untuk setiap destinasi menggunakan TF-IDF dan Word2Vec.
# 4. Menggabungkan kedua representasi menjadi sebuah *hybrid vector*.
# 5. Menghitung matriks kemiripan baru dari *hybrid vector* tersebut.
# 6. Menyimpan matriks kemiripan hybrid untuk digunakan di aplikasi.

# %%
# ======================================================
# 1️⃣ SETUP & IMPORT LIBRARY
# ======================================================
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize
import pickle
from tqdm import tqdm
from pathlib import Path
import logging

# Konfigurasi logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] - %(message)s")

# Konfigurasi path yang robust
# Karena notebook ini ada di dalam folder 'notebooks/', .parent akan menunjuk ke root proyek
BASE_DIR = Path().resolve().parent 
DATA_PATH = BASE_DIR / "data" / "processed" / "destinasi_processed.csv"
MODELS_DIR = BASE_DIR / "models"
TFIDF_VECTORIZER_PATH = MODELS_DIR / "tfidf_vectorizer.pkl"
HYBRID_SIM_PATH = MODELS_DIR / "hybrid_similarity.pkl"

logging.info("✅ Library dan path siap digunakan.")

2025-10-18 02:16:33,881 [INFO] - ✅ Library dan path siap digunakan.


Memuat Artefak & Dataset

In [2]:
# %% [markdown]
# ## 2. Memuat Dataset dan Artefak dari Fase 1
# 
# Kita akan memuat dataset yang sudah bersih dan TF-IDF Vectorizer yang sudah dilatih dari notebook sebelumnya.

# %%
# ======================================================
# 2️⃣ MEMUAT DATASET & TF-IDF VECTORIZER
# ======================================================
try:
    with open(TFIDF_VECTORIZER_PATH, "rb") as f:
        tfidf_vectorizer = pickle.load(f)
    df = pd.read_csv(DATA_PATH)
    df['fitur_bersih'] = df['fitur_bersih'].fillna('') # Jaring pengaman
    
    logging.info(f"✅ Artefak TF-IDF dan dataset ({len(df)} baris) berhasil dimuat.")
    display(df.head(2))
except FileNotFoundError as e:
    raise FileNotFoundError(f"❌ File tidak ditemukan: {e}. Pastikan Anda sudah menjalankan notebook Fase 1 & 2.")

2025-10-18 02:16:46,293 [INFO] - ✅ Artefak TF-IDF dan dataset (55 baris) berhasil dimuat.


,id,nama_wisata,kategori,kota,alamat,deskripsi,gambar,fitur,fitur_bersih
0,1,Puncak Rembangan Resort,Rekreasi,Jember,"Darungan, Kemuning Lor, Kec. Arjasa, Kabupaten...",Puncak Rembangan adalah destinasi wisata pegun...,assets/images/1.png,Puncak Rembangan Resort Rekreasi Rekreasi Jemb...,puncak rembangan resort rekreasi rekreasi jemb...
1,2,Pantai Watu Ulo,Pantai,Jember,"Sumberrejo, Kec. Ambulu, Kabupaten Jember, Jaw...",Pantai Watu Ulo terkenal karena batu karang pa...,assets/images/2.png,Pantai Watu Ulo Pantai Pantai Jember Sumberrej...,pantai watu ulo pantai pantai jember sumberrej...


Memuat Model Word2Vec (FastText)

In [3]:
# %% [markdown]
# ## 3. Memuat Model Word2Vec (FastText) Bahasa Indonesia
# 
# Kita akan menggunakan model pre-trained dari Facebook (FastText) yang sudah dilatih pada data teks Bahasa Indonesia yang sangat besar. Ini akan memberikan pemahaman semantik yang mendalam pada model kita.

# %%
# ======================================================
# 3️⃣ MEMUAT WORD2VEC (FASTTEXT)
# ======================================================
W2V_MODEL_PATH = MODELS_DIR / "fasttext_cc.id.300.vec.gz" # Path lokal untuk menyimpan model
W2V_DOWNLOAD_URL = "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.id.300.vec.gz"

if not W2V_MODEL_PATH.exists():
    logging.warning("⚠️ Model Word2Vec lokal tidak ditemukan. Memulai proses unduh (~1GB)...")
    # Anda bisa mengunduhnya manual dan meletakkannya di folder models,
    # atau menjalankan kode ini jika koneksi internet stabil.
    # import requests
    # with requests.get(W2V_DOWNLOAD_URL, stream=True) as r:
    #     r.raise_for_status()
    #     with open(W2V_MODEL_PATH, 'wb') as f:
    #         for chunk in r.iter_content(chunk_size=8192):
    #             f.write(chunk)
    # logging.info("✅ Model Word2Vec berhasil diunduh.")
    raise FileNotFoundError(f"Harap unduh model dari {W2V_DOWNLOAD_URL} dan letakkan di {W2V_MODEL_PATH}")

logging.info("Memuat model Word2Vec dari penyimpanan lokal (mungkin butuh beberapa menit)...")
word2vec_model = KeyedVectors.load_word2vec_format(W2V_MODEL_PATH, binary=False)
logging.info("✅ Model Word2Vec berhasil dimuat.")

2025-10-18 02:16:58,635 [WARNING] - ⚠️ Model Word2Vec lokal tidak ditemukan. Memulai proses unduh (~1GB)...


FileNotFoundError: Harap unduh model dari https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.id.300.vec.gz dan letakkan di D:\iann Kuliah\Semester 7\1. Artificial Intelegence (AI)\wisata-recommender\models\fasttext_cc.id.300.vec.gz

Membuat Vektor Rata-rata Word2Vec

In [ ]:
# %% [markdown]
# ## 4. Membuat Vektor Rata-rata Word2Vec untuk Setiap Destinasi
# 
# Kita akan mengubah setiap teks 'fitur_bersih' menjadi satu vektor tunggal dengan cara merata-ratakan vektor dari semua kata yang ada di dalamnya.

# %%
# ======================================================
# 4️⃣ MEMBUAT VEKTOR WORD2VEC PER DOKUMEN
# ======================================================
def get_sentence_vector(sentence, model, dimensions=300):
    """Mengubah sebuah kalimat menjadi vektor rata-rata dari kata-katanya."""
    words = [word for word in sentence.split() if word in model.key_to_index]
    if not words:
        return np.zeros(dimensions)
    return np.mean(model[words], axis=0)

# Terapkan fungsi ke setiap baris di DataFrame
tqdm.pandas(desc="Membuat vektor Word2Vec")
word2vec_matrix = np.array(df['fitur_bersih'].progress_apply(
    lambda x: get_sentence_vector(x, word2vec_model)
).tolist())

logging.info(f"✅ Matriks Word2Vec berhasil dibuat dengan ukuran: {word2vec_matrix.shape}")

Menggabungkan Vektor & Menghitung Kemiripan

In [ ]:
# %% [markdown]
# ## 5. Membuat Vektor Hybrid dan Menghitung Cosine Similarity
# 
# Ini adalah inti dari V2.0. Kita akan menggabungkan kekuatan TF-IDF (yang bagus untuk kata kunci) dan Word2Vec (yang bagus untuk makna) menjadi satu vektor hybrid.

# %%
# ======================================================
# 5️⃣ MEMBUAT VEKTOR HYBRID & MENGHITUNG KEMIRIPAN
# ======================================================
# 1. Buat matriks TF-IDF dari data yang sama
# ⭐ PERBAIKAN KRITIS: Gunakan 'fitur_bersih', bukan 'deskripsi'
logging.info("Membuat matriks TF-IDF...")
tfidf_matrix = tfidf_vectorizer.transform(df['fitur_bersih']).toarray()

# 2. Normalisasi kedua matriks agar skalanya seimbang
logging.info("Normalisasi matriks TF-IDF dan Word2Vec...")
tfidf_matrix = normalize(tfidf_matrix)
word2vec_matrix = normalize(word2vec_matrix)

# 3. Gabungkan kedua matriks dengan bobot (alpha)
alpha = 0.6  # Beri bobot lebih pada TF-IDF (kata kunci)
hybrid_matrix = (alpha * tfidf_matrix) + ((1 - alpha) * word2vec_matrix)
logging.info(f"✅ Matriks Hybrid berhasil dibuat dengan bobot alpha={alpha}")

# 4. Hitung matriks cosine similarity dari vektor hybrid
hybrid_similarity_matrix = cosine_similarity(hybrid_matrix)
logging.info(f"✅ Matriks kemiripan hybrid berhasil dihitung dengan ukuran: {hybrid_similarity_matrix.shape}")

Menyimpan dan Menguji Model Hybrid

In [ ]:
# %% [markdown]
# ## 6. Menyimpan Matriks Kemiripan Hybrid
# 
# Simpan matriks baru ini agar bisa digunakan oleh `recommender.py` dan `app.py`.

# %%
# ======================================================
# 6️⃣ MENYIMPAN MATRIKS HYBRID
# ======================================================
with open(HYBRID_SIM_PATH, "wb") as f:
    pickle.dump(hybrid_similarity_matrix, f)
logging.info(f"💾 Matriks kemiripan hybrid berhasil disimpan di: {HYBRID_SIM_PATH}")

# %% [markdown]
# ## 7. Pengujian Cepat
# 
# Mari kita lihat apakah model hybrid memberikan rekomendasi yang berbeda atau lebih baik dari model TF-IDF murni.

# %%
# ======================================================
# 7️⃣ PENGUJIAN CEPAT
# ======================================================
# Pilih destinasi referensi
nama_wisata_ref = "Pantai Papuma"

# Dapatkan indeksnya
idx_ref = df[df['nama_wisata'] == nama_wisata_ref].index[0]

# Ambil skor kemiripan dan urutkan
sim_scores = list(enumerate(hybrid_similarity_matrix[idx_ref]))
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:6]

print(f"\n🎯 5 Rekomendasi Hybrid untuk '{nama_wisata_ref}':\n")
for i, score in sim_scores:
    print(f"- {df.iloc[i]['nama_wisata']} (Skor: {score:.3f})")